### Quickstart: Compare runs, choose a model, and deploy it to a REST API
- run hyperparameter sweep on a training script
- choose the best run and register it as a model
- compare results in mlflow
- deploy model to REST API
- Build a container image suitable for deployment to a cloud platform


In [33]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,Trials,fmin,hp,tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

In [34]:
# load dataset

data=pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",sep=";")


In [35]:
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


In [36]:
train,test=train_test_split(data,test_size=0.25,random_state=40)

In [37]:
train_x = train.drop(['quality'], axis=1).values
train_y=train['quality'].values.ravel()

## validation dataset
test_x=test.drop(['quality'], axis=1).values
test_y=test['quality'].values.ravel()

## splitting train into train and validation
train_x,valid_x,train_y,valid_y=train_test_split(train_x,train_y,test_size=0.20,random_state=40)

signature = infer_signature(train_x,train_y)

In [38]:
### ANN Model

def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):

    ## Define model architechture
    mean = np.mean(train_x,axis=0)
    var=np.var(train_x,axis=0)

    model=keras.Sequential(
        [
            keras.Input([train_x.shape[1]]),
            keras.layers.Normalization(mean=mean,variance=var),
            keras.layers.Dense(64,activation='relu'),
            keras.layers.Dense(1)
        ]
    )

    ## compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params["lr"],momentum=params["momentum"]
    ),
    loss="mean_squared_error",
    metrics=[keras.metrics.RootMeanSquaredError()]
    )

    ## Train the ANN model with lr and momentum params with mlflow tracking
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),epochs=epochs,batch_size=64)

        ## Evaluate model
        eval_result=model.evaluate(valid_x,valid_y,batch_size=64)

        eval_rmse=eval_result[1]

        ## Log the parameters and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse)

        ## log the model

        mlflow.tensorflow.log_model(model,"model",signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}





In [39]:
space={
    "lr":hp.loguniform("lr",np.log(1e-5),np.log(1e-1)),
    "momentum": hp.uniform("momentum",0.0,1.0)
}

In [40]:
def objective(params):
    result = train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y,
    )
    return result

In [41]:
mlflow.set_experiment("/wine-quality")
with mlflow.start_run():
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse",best_run["loss"])
    mlflow.tensorflow.log_model(best_run["model"],"model",signature=signature)

    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_run['loss']}")

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

Epoch 1/3                                            

  0%|          | 0/4 [00:00<?, ?trial/s, best loss=?]

2024-12-04 17:31:38.599605: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


 1/46 ━━━━━━━━━━━━━━━━━━━━ 26s 583ms/step - loss: 39.3736 - root_mean_squared_error: 6.2748
29/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 8.1985 - root_mean_squared_error: 2.6654    
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 6.0678 - root_mean_squared_error: 2.2657 - val_loss: 0.7097 - val_root_mean_squared_error: 0.8424

Epoch 2/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.7356 - root_mean_squared_error: 0.8577
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.6848 - root_mean_squared_error: 0.8268 - val_loss: 0.5705 - val_root_mean_squared_error: 0.7553

Epoch 3/3                                            

 1/46 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.4454 - root_mean_squared_error: 0.6674
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.5465 - root_mean_squared_error: 0.7391 - val_loss: 0.5796 - val_root_mean_squared_error: 0.7613

 1/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 0.6506 - root_mean_squared_error: 0.8066
1

In [ ]:
#Inferencing
from mlflow.models import validate_serving_input

model_uri = 'runs:/63a9815e1e0c462b80839ec560bd427c/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_x)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm

 1/39 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step

39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


array([[5.873934 ],
       [4.463093 ],
       [5.838876 ],
       ...,
       [5.5656157],
       [5.964863 ],
       [5.7269087]], dtype=float32)

In [43]:
# Register model in regestry
mlflow.register_model(model_uri,"wine-quality")

Successfully registered model 'wine-quality'.
Created version '1' of model 'wine-quality'.


<ModelVersion: aliases=[], creation_timestamp=1733333823733, current_stage='None', description=None, last_updated_timestamp=1733333823733, name='wine-quality', run_id='63a9815e1e0c462b80839ec560bd427c', run_link=None, source='file:///workspaces/mlflow-integration/wine-mlflow/mlruns/530757564646479595/63a9815e1e0c462b80839ec560bd427c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>